# MAPS: Progressive 3D Model Transmission
## Multiresolution Adaptive Parameterization of Surfaces

**Group Members:** [Name 1], [Name 2], [Name 3], [Name 4]

### Project Overview
Implementation of MAPS algorithm inverted for progressive transmission.
- **Input:** Dense OBJ mesh
- **Output:** Progressive OBJA format
- **Method:** DK hierarchy → Parameterization → Progressive encoding

## 1. Setup and Imports

In [ ]:
# Install required packages
!pip install numpy scipy matplotlib trimesh networkx

# Clone the OBJA repository
!git clone https://gitea.tforgione.fr/tforgione/obja.git
%cd obja

In [ ]:
import numpy as np
import heapq
from dataclasses import dataclass
from typing import List, Tuple, Dict, Set, Optional
from collections import defaultdict
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
import math

# Import provided OBJA parser
from obja import parse_file, Output

## 2. Data Structures

### Core mesh representation and hierarchy management

In [ ]:
@dataclass
class Vertex:
    """3D vertex with index"""
    id: int
    x: float
    y: float
    z: float
    
    def position(self):
        return np.array([self.x, self.y, self.z])
    
    def __repr__(self):
        return f"v{self.id}({self.x:.2f}, {self.y:.2f}, {self.z:.2f})"

@dataclass
class Face:
    """Triangular face with vertex indices"""
    v1: int
    v2: int
    v3: int
    
    def vertices(self):
        return [self.v1, self.v2, self.v3]
    
    def __repr__(self):
        return f"f({self.v1}, {self.v2}, {self.v3})"

@dataclass
class BarycentricCoord:
    """Barycentric coordinates of a vertex within a triangle"""
    triangle_id: int  # Which triangle in the coarser level
    alpha: float
    beta: float
    gamma: float
    
    def __post_init__(self):
        # Ensure coordinates sum to 1
        total = self.alpha + self.beta + self.gamma
        assert abs(total - 1.0) < 1e-6, f"Barycentric coords must sum to 1, got {total}"

class MeshLevel:
    """One level in the mesh hierarchy"""
    def __init__(self, level: int):
        self.level = level
        self.vertices: Dict[int, Vertex] = {}  # id -> Vertex
        self.faces: List[Face] = []
        self.removed_vertices: List[int] = []  # Vertices removed to get to next coarser level
        self.barycentric_map: Dict[int, BarycentricCoord] = {}  # Maps vertex to its position in coarser level
        
    def add_vertex(self, v: Vertex):
        self.vertices[v.id] = v
    
    def add_face(self, f: Face):
        self.faces.append(f)
    
    def num_vertices(self):
        return len(self.vertices)
    
    def num_faces(self):
        return len(self.faces)

class MeshHierarchy:
    """Complete hierarchy from fine to coarse"""
    def __init__(self):
        self.levels: List[MeshLevel] = []
        self.finest_level: int = 0
        self.coarsest_level: int = 0
    
    def add_level(self, level: MeshLevel):
        self.levels.append(level)
    
    def get_level(self, l: int) -> MeshLevel:
        return self.levels[l]
    
    def num_levels(self):
        return len(self.levels)

## 3. Mesh Topology and Adjacency

### Managing 1-rings, neighbors, and connectivity

In [ ]:
class MeshTopology:
    """Manages mesh connectivity and adjacency information"""
    
    def __init__(self, vertices: Dict[int, Vertex], faces: List[Face]):
        self.vertices = vertices
        self.faces = faces
        self.adjacency = self._build_adjacency()
        self.vertex_faces = self._build_vertex_faces()
    
    def _build_adjacency(self) -> Dict[int, Set[int]]:
        """Build vertex adjacency (1-ring neighbors)"""
        adj = defaultdict(set)
        for face in self.faces:
            v1, v2, v3 = face.vertices()
            adj[v1].update([v2, v3])
            adj[v2].update([v1, v3])
            adj[v3].update([v1, v2])
        return adj
    
    def _build_vertex_faces(self) -> Dict[int, List[int]]:
        """Map each vertex to faces containing it"""
        vf = defaultdict(list)
        for i, face in enumerate(self.faces):
            for v in face.vertices():
                vf[v].append(i)
        return vf
    
    def get_neighbors(self, vertex_id: int) -> List[int]:
        """Get 1-ring neighbors of a vertex"""
        return list(self.adjacency[vertex_id])
    
    def get_vertex_degree(self, vertex_id: int) -> int:
        """Get outdegree (number of neighbors)"""
        return len(self.adjacency[vertex_id])
    
    def get_star(self, vertex_id: int) -> List[Face]:
        """Get all faces incident to a vertex (star)"""
        return [self.faces[i] for i in self.vertex_faces[vertex_id]]
    
    def get_1ring_ordered(self, vertex_id: int) -> List[int]:
        """Get 1-ring neighbors in cyclic order"""
        # TODO: Implement cyclic ordering of neighbors
        # This is needed for conformal mapping
        neighbors = self.get_neighbors(vertex_id)
        star_faces = self.get_star(vertex_id)
        
        # Order neighbors by traversing faces
        ordered = []
        # Implementation needed
        
        return neighbors  # Placeholder
    
    def is_boundary_vertex(self, vertex_id: int) -> bool:
        """Check if vertex is on mesh boundary"""
        # TODO: Implement boundary detection
        return False
    
    def find_independent_set(self, max_degree: int = 12) -> Set[int]:
        """Find maximally independent set with degree constraint"""
        independent = set()
        marked = set()
        
        # Get vertices sorted by degree
        vertices_by_degree = sorted(
            [v for v in self.vertices.keys() if self.get_vertex_degree(v) <= max_degree],
            key=lambda v: self.get_vertex_degree(v)
        )
        
        for v in vertices_by_degree:
            if v not in marked:
                independent.add(v)
                marked.add(v)
                # Mark all neighbors
                for neighbor in self.get_neighbors(v):
                    marked.add(neighbor)
        
        return independent

## 4. Geometry Utilities

### Member 1: Conformal mapping, curvature, area computation

In [ ]:
class GeometryUtils:
    """Geometric computations for MAPS algorithm"""
    
    @staticmethod
    def compute_area_1ring(center: Vertex, neighbors: List[Vertex], topology: MeshTopology) -> float:
        """
        Compute area of 1-ring neighborhood
        TODO: Implement area computation from star faces
        """
        area = 0.0
        star_faces = topology.get_star(center.id)
        
        for face in star_faces:
            # Get triangle vertices
            # Compute triangle area
            pass
        
        return area
    
    @staticmethod
    def estimate_curvature(center: Vertex, neighbors: List[Vertex], topology: MeshTopology) -> float:
        """
        Conservative curvature estimate: κ = |κ1| + |κ2|
        Uses tangent plane and second-degree polynomial approximation
        TODO: Implement curvature estimation
        """
        # Establish tangent plane at center
        # Fit second-degree polynomial
        # Extract principal curvatures κ1, κ2
        
        curvature = 0.0
        return curvature
    
    @staticmethod
    def conformal_flatten_1ring(center: Vertex, neighbors: List[Vertex]) -> List[Tuple[float, float]]:
        """
        Map 1-ring to plane using conformal map z^a
        
        Returns list of 2D coordinates for each neighbor
        
        Algorithm from MAPS paper:
        - μ(center) = 0 (origin)
        - μ(neighbor_k) = r_k * exp(i * θ_k^a)
        - r_k = ||center - neighbor_k||
        - θ_k = sum of angles from neighbor_0 to neighbor_k
        - a = 2π / θ_total (normalization factor)
        """
        K = len(neighbors)
        flattened = []
        
        # Compute angles between consecutive neighbors
        angles = []
        for k in range(K):
            # Angle at center between neighbor_k-1, center, neighbor_k
            v1 = neighbors[k-1].position() - center.position()
            v2 = neighbors[k].position() - center.position()
            
            # Compute angle
            cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            cos_angle = np.clip(cos_angle, -1.0, 1.0)
            angle = np.arccos(cos_angle)
            angles.append(angle)
        
        theta_total = sum(angles)
        a = 2 * np.pi / theta_total  # Conformal scaling factor
        
        # Map each neighbor to plane
        theta = 0
        for k in range(K):
            r_k = np.linalg.norm(neighbors[k].position() - center.position())
            
            # Conformal map: z = r * exp(i * a * theta)
            x = r_k * np.cos(a * theta)
            y = r_k * np.sin(a * theta)
            
            flattened.append((x, y))
            theta += angles[k]
        
        return flattened
    
    @staticmethod
    def conformal_flatten_boundary(center: Vertex, neighbors: List[Vertex]) -> List[Tuple[float, float]]:
        """
        Map boundary vertex 1-ring to half-disk
        Uses a = π / θ_total instead of 2π / θ_total
        """
        # Similar to conformal_flatten_1ring but with a = π / θ_total
        # TODO: Implement boundary case
        return []
    
    @staticmethod
    def retriangulate_hole(flattened_points: List[Tuple[float, float]]) -> List[Tuple[int, int, int]]:
        """
        Retriangulate hole using Constrained Delaunay Triangulation
        Input: 2D points in plane (after conformal flattening)
        Output: Triangle indices
        """
        if len(flattened_points) < 3:
            return []
        
        points = np.array(flattened_points)
        
        # Perform Delaunay triangulation
        try:
            tri = Delaunay(points)
            triangles = tri.simplices.tolist()
            return triangles
        except:
            # Fallback: simple fan triangulation
            triangles = []
            for i in range(1, len(flattened_points) - 1):
                triangles.append((0, i, i+1))
            return triangles
    
    @staticmethod
    def compute_barycentric_coords(point: np.ndarray, 
                                   v1: np.ndarray, 
                                   v2: np.ndarray, 
                                   v3: np.ndarray) -> Tuple[float, float, float]:
        """
        Compute barycentric coordinates of point within triangle (v1, v2, v3)
        Returns (alpha, beta, gamma) such that:
        point = alpha * v1 + beta * v2 + gamma * v3
        alpha + beta + gamma = 1
        """
        v0 = v2 - v1
        v1_vec = v3 - v1
        v2_vec = point - v1
        
        dot00 = np.dot(v0, v0)
        dot01 = np.dot(v0, v1_vec)
        dot02 = np.dot(v0, v2_vec)
        dot11 = np.dot(v1_vec, v1_vec)
        dot12 = np.dot(v1_vec, v2_vec)
        
        inv_denom = 1 / (dot00 * dot11 - dot01 * dot01)
        gamma = (dot11 * dot02 - dot01 * dot12) * inv_denom
        beta = (dot00 * dot12 - dot01 * dot02) * inv_denom
        alpha = 1.0 - beta - gamma
        
        return (alpha, beta, gamma)
    
    @staticmethod
    def point_in_triangle_2d(point: Tuple[float, float],
                            v1: Tuple[float, float],
                            v2: Tuple[float, float],
                            v3: Tuple[float, float]) -> bool:
        """
        Check if 2D point is inside 2D triangle
        """
        def sign(p1, p2, p3):
            return (p1[0] - p3[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p3[1])
        
        d1 = sign(point, v1, v2)
        d2 = sign(point, v2, v3)
        d3 = sign(point, v3, v1)
        
        has_neg = (d1 < 0) or (d2 < 0) or (d3 < 0)
        has_pos = (d1 > 0) or (d2 > 0) or (d3 > 0)
        
        return not (has_neg and has_pos)

## 5. Mesh Simplification (DK Hierarchy)

### Member 1: Hierarchy construction with priority queue

In [ ]:
class MeshSimplifier:
    """Dobkin-Kirkpatrick hierarchy construction"""
    
    def __init__(self, lambda_weight: float = 0.5, max_degree: int = 12):
        self.lambda_weight = lambda_weight
        self.max_degree = max_degree
        self.geom_utils = GeometryUtils()
    
    def compute_vertex_priority(self, vertex: Vertex, topology: MeshTopology, 
                               max_area: float, max_curvature: float) -> float:
        """
        Compute removal priority for vertex
        Lower priority = remove first
        
        w(λ, i) = λ * a(i)/max_a + (1-λ) * κ(i)/max_κ
        """
        neighbors = [topology.vertices[n] for n in topology.get_neighbors(vertex.id)]
        
        area = self.geom_utils.compute_area_1ring(vertex, neighbors, topology)
        curvature = self.geom_utils.estimate_curvature(vertex, neighbors, topology)
        
        # Normalize
        norm_area = area / max_area if max_area > 0 else 0
        norm_curv = curvature / max_curvature if max_curvature > 0 else 0
        
        priority = self.lambda_weight * norm_area + (1 - self.lambda_weight) * norm_curv
        
        return priority
    
    def build_hierarchy(self, mesh_level: MeshLevel) -> MeshHierarchy:
        """
        Build complete DK hierarchy from finest to coarsest
        
        Returns MeshHierarchy with L levels (L ~ log N)
        """
        hierarchy = MeshHierarchy()
        hierarchy.finest_level = 0
        
        current_level = mesh_level
        level_idx = 0
        
        # Add finest level
        hierarchy.add_level(current_level)
        
        # Simplify until base domain is small enough
        while current_level.num_vertices() > 50:  # Stop when ~50 vertices remain
            print(f"Level {level_idx}: {current_level.num_vertices()} vertices")
            
            # Simplify one level
            coarser_level = self.simplify_one_level(current_level, level_idx + 1)
            
            hierarchy.add_level(coarser_level)
            current_level = coarser_level
            level_idx += 1
            
            # Safety check
            if level_idx > 30:
                print("Warning: Too many levels, stopping")
                break
        
        hierarchy.coarsest_level = level_idx
        print(f"Hierarchy complete: {level_idx + 1} levels")
        
        return hierarchy
    
    def simplify_one_level(self, mesh_level: MeshLevel, new_level_idx: int) -> MeshLevel:
        """
        Perform one DK simplification step:
        1. Find maximally independent set with low degree
        2. Remove vertices using priority queue
        3. Retriangulate holes with conformal mapping
        4. Track barycentric coordinates
        """
        topology = MeshTopology(mesh_level.vertices, mesh_level.faces)
        
        # Compute max area and curvature for normalization
        max_area = 0.0
        max_curvature = 0.0
        # TODO: Compute these properly
        
        # Build priority queue of vertices to remove
        priority_queue = []
        for vid, vertex in mesh_level.vertices.items():
            degree = topology.get_vertex_degree(vid)
            if degree <= self.max_degree and not topology.is_boundary_vertex(vid):
                priority = self.compute_vertex_priority(vertex, topology, max_area, max_curvature)
                heapq.heappush(priority_queue, (priority, vid))
        
        # Select independent set to remove
        to_remove = set()
        marked = set()
        
        while priority_queue:
            priority, vid = heapq.heappop(priority_queue)
            
            if vid not in marked:
                to_remove.add(vid)
                marked.add(vid)
                # Mark all neighbors
                for neighbor in topology.get_neighbors(vid):
                    marked.add(neighbor)
        
        print(f"  Removing {len(to_remove)} vertices ({100*len(to_remove)/mesh_level.num_vertices():.1f}%)")
        
        # Create coarser level
        coarser_level = MeshLevel(new_level_idx)
        
        # TODO: Implement vertex removal and retriangulation
        # For each vertex in to_remove:
        #   1. Get ordered 1-ring neighbors
        #   2. Flatten 1-ring using conformal map
        #   3. Retriangulate hole
        #   4. Add new faces to coarser level
        #   5. Store barycentric coordinates for removed vertex
        
        # Copy vertices not removed
        for vid, vertex in mesh_level.vertices.items():
            if vid not in to_remove:
                coarser_level.add_vertex(vertex)
        
        # Store removed vertices
        mesh_level.removed_vertices = list(to_remove)
        
        return coarser_level

## 6. Parameterization Construction

### Member 2: Build mapping Π from fine to coarse

In [ ]:
class ParameterizationBuilder:
    """Build parameterization Π: fine mesh → base domain"""
    
    def __init__(self):
        self.geom_utils = GeometryUtils()
    
    def build_parameterization(self, hierarchy: MeshHierarchy) -> Dict[int, BarycentricCoord]:
        """
        Build complete parameterization from hierarchy
        
        Returns mapping from finest level vertex id to barycentric coords in base domain
        
        Algorithm:
        - Start with Π_L = identity (finest level)
        - For each level l from L to 1:
          - Update Π_{l-1} based on vertex removals
        - Result: Π_0 maps all original vertices to base domain
        """
        L = hierarchy.num_levels() - 1
        
        # Initialize: finest level vertices map to themselves
        finest_level = hierarchy.get_level(L)
        parameterization = {}
        
        # Process each level from fine to coarse
        for l in range(L, 0, -1):
            current_level = hierarchy.get_level(l)
            coarser_level = hierarchy.get_level(l - 1)
            
            print(f"Building parameterization for level {l}")
            
            # Update parameterization based on removed vertices
            self._update_parameterization(parameterization, current_level, coarser_level)
        
        return parameterization
    
    def _update_parameterization(self, parameterization: Dict[int, BarycentricCoord],
                                current_level: MeshLevel,
                                coarser_level: MeshLevel):
        """
        Update parameterization when going from level l to l-1
        
        Three cases for each vertex i in finest level:
        1. i ∈ K_{l-1}: vertex survives, Π_{l-1}(i) = Π_l(i)
        2. i ∈ K_l \ K_{l-1}: vertex removed, compute new barycentric coords
        3. i ∈ K_L \ K_l: vertex removed earlier, remap through new triangulation
        """
        # TODO: Implement parameterization update
        pass
    
    def find_containing_triangle(self, point_2d: Tuple[float, float],
                                triangles: List[Face],
                                vertices_2d: Dict[int, Tuple[float, float]]) -> Optional[Face]:
        """
        Find which triangle contains a 2D point
        Used after retriangulation to assign barycentric coordinates
        """
        for triangle in triangles:
            v1_2d = vertices_2d[triangle.v1]
            v2_2d = vertices_2d[triangle.v2]
            v3_2d = vertices_2d[triangle.v3]
            
            if self.geom_utils.point_in_triangle_2d(point_2d, v1_2d, v2_2d, v3_2d):
                return triangle
        
        return None

## 7. Progressive Encoding

### Member 3: Invert hierarchy and generate OBJA

In [ ]:
class ProgressiveEncoder:
    """Generate progressive OBJA format from hierarchy"""
    
    def __init__(self):
        pass
    
    def encode(self, hierarchy: MeshHierarchy, 
              parameterization: Dict[int, BarycentricCoord],
              output_path: str):
        """
        Generate OBJA file for progressive transmission
        
        Strategy:
        1. Start with base domain (coarsest level)
        2. Progressively add vertices from coarse to fine
        3. Order vertices by geometric importance
        4. Generate OBJA packets
        """
        output = Output(output_path, random_color=True)
        
        # Level 0: Base domain
        base_level = hierarchy.get_level(0)
        self._encode_base_level(base_level, output)
        
        # Levels 1 to L: Progressive refinement
        for l in range(1, hierarchy.num_levels()):
            level = hierarchy.get_level(l)
            prev_level = hierarchy.get_level(l - 1)
            
            print(f"Encoding level {l}")
            self._encode_level(level, prev_level, output)
        
        output.close()
        print(f"OBJA file written to {output_path}")
    
    def _encode_base_level(self, base_level: MeshLevel, output: Output):
        """
        Encode base domain (coarsest level)
        """
        # Add all base vertices
        for vid, vertex in sorted(base_level.vertices.items()):
            output.add_vertex(vertex.x, vertex.y, vertex.z)
        
        # Add all base faces
        for face in base_level.faces:
            output.add_face(face.v1, face.v2, face.v3)
    
    def _encode_level(self, level: MeshLevel, prev_level: MeshLevel, output: Output):
        """
        Encode one refinement level
        
        Adds:
        - New vertices (those removed when creating prev_level)
        - New faces (retriangulation)
        """
        # Find vertices that were removed
        new_vertices = [vid for vid in level.vertices.keys() 
                       if vid not in prev_level.vertices]
        
        # Sort by geometric importance (optional: use priority)
        new_vertices.sort()
        
        # Add new vertices
        for vid in new_vertices:
            vertex = level.vertices[vid]
            output.add_vertex(vertex.x, vertex.y, vertex.z)
        
        # TODO: Add new faces
        # This requires tracking which faces were modified
    
    def compute_approximation_error(self, level: MeshLevel, 
                                   finest_level: MeshLevel,
                                   parameterization: Dict[int, BarycentricCoord]) -> float:
        """
        Compute Hausdorff distance between current level and finest level
        
        E(t) = max_{p_i ∈ P_L, Π(p_i) ∈ φ(|t|)} dist(p_i, φ(|t|))
        """
        # TODO: Implement error computation
        return 0.0

## 8. Main Pipeline

### Member 4: Integration of all components

In [ ]:
class MAPSPipeline:
    """Complete MAPS pipeline for progressive transmission"""
    
    def __init__(self):
        self.simplifier = MeshSimplifier()
        self.param_builder = ParameterizationBuilder()
        self.encoder = ProgressiveEncoder()
    
    def process(self, input_obj_path: str, output_obja_path: str):
        """
        Complete pipeline:
        OBJ input → Hierarchy → Parameterization → OBJA output
        """
        print("=== MAPS Progressive Transmission Pipeline ===")
        
        # Step 1: Load input mesh
        print("\n[1] Loading input mesh...")
        finest_level = self.load_mesh(input_obj_path)
        print(f"    Loaded: {finest_level.num_vertices()} vertices, {finest_level.num_faces()} faces")
        
        # Step 2: Build DK hierarchy
        print("\n[2] Building DK hierarchy...")
        hierarchy = self.simplifier.build_hierarchy(finest_level)
        print(f"    Created {hierarchy.num_levels()} levels")
        
        # Step 3: Build parameterization
        print("\n[3] Building parameterization...")
        parameterization = self.param_builder.build_parameterization(hierarchy)
        print(f"    Parameterized {len(parameterization)} vertices")
        
        # Step 4: Encode to OBJA
        print("\n[4] Encoding to OBJA format...")
        self.encoder.encode(hierarchy, parameterization, output_obja_path)
        print(f"    Output written to {output_obja_path}")
        
        print("\n=== Pipeline complete ===")
    
    def load_mesh(self, obj_path: str) -> MeshLevel:
        """
        Load OBJ file using provided parser
        """
        # Use provided OBJA parser
        model = parse_file(obj_path)
        
        # Convert to our format
        level = MeshLevel(level=0)
        
        for i, vertex in enumerate(model.vertices):
            v = Vertex(id=i, x=vertex.x, y=vertex.y, z=vertex.z)
            level.add_vertex(v)
        
        for face in model.faces:
            # Note: OBJ indices are 1-based, convert to 0-based
            f = Face(v1=face.a.vertex - 1, 
                    v2=face.b.vertex - 1, 
                    v3=face.c.vertex - 1)
            level.add_face(f)
        
        return level

## 9. Testing

### Member 4: Unit and integration tests

In [ ]:
def test_conformal_mapping():
    """Test conformal flattening of 1-ring"""
    print("Testing conformal mapping...")
    
    # Create simple test case: vertex at origin with 6 neighbors
    center = Vertex(0, 0.0, 0.0, 0.0)
    neighbors = [
        Vertex(1, 1.0, 0.0, 0.0),
        Vertex(2, 0.5, 0.866, 0.0),
        Vertex(3, -0.5, 0.866, 0.0),
        Vertex(4, -1.0, 0.0, 0.0),
        Vertex(5, -0.5, -0.866, 0.0),
        Vertex(6, 0.5, -0.866, 0.0),
    ]
    
    flattened = GeometryUtils.conformal_flatten_1ring(center, neighbors)
    
    # Verify: should get 6 points roughly in a circle
    assert len(flattened) == 6
    
    # Check angles between consecutive points
    for i in range(6):
        angle = math.atan2(flattened[i][1], flattened[i][0])
        expected_angle = i * math.pi / 3
        # Allow some tolerance
        assert abs(angle - expected_angle) < 0.1, f"Angle mismatch at {i}: {angle} vs {expected_angle}"
    
    print("  ✓ Conformal mapping test passed")

def test_barycentric_coords():
    """Test barycentric coordinate computation"""
    print("Testing barycentric coordinates...")
    
    v1 = np.array([0.0, 0.0, 0.0])
    v2 = np.array([1.0, 0.0, 0.0])
    v3 = np.array([0.0, 1.0, 0.0])
    
    # Test center of triangle
    center = (v1 + v2 + v3) / 3
    alpha, beta, gamma = GeometryUtils.compute_barycentric_coords(center, v1, v2, v3)
    
    assert abs(alpha - 1/3) < 0.001
    assert abs(beta - 1/3) < 0.001
    assert abs(gamma - 1/3) < 0.001
    assert abs(alpha + beta + gamma - 1.0) < 0.001
    
    print("  ✓ Barycentric coordinates test passed")

def test_full_pipeline():
    """Test complete pipeline on simple mesh"""
    print("Testing full pipeline...")
    
    # Use example cube from repository
    pipeline = MAPSPipeline()
    
    try:
        pipeline.process('example/example.obj', 'output/test_output.obja')
        print("  ✓ Full pipeline test passed")
    except Exception as e:
        print(f"  ✗ Full pipeline test failed: {e}")

# Run all tests
def run_all_tests():
    print("\n=== Running Tests ===")
    test_conformal_mapping()
    test_barycentric_coords()
    test_full_pipeline()
    print("=== Tests Complete ===")

## 10. Example Usage

In [ ]:
# Run tests first
run_all_tests()

In [ ]:
# Process an example mesh
pipeline = MAPSPipeline()
pipeline.process(
    input_obj_path='example/example.obj',
    output_obja_path='output/progressive.obja'
)

In [ ]:
# Visualize results with provided server
# Run this in terminal:
# python server.py output/progressive.obja

## 11. Performance Evaluation

### Evaluate on online platform: https://csi.alcouffe.eu/

In [ ]:
def evaluate_compression_efficiency(hierarchy: MeshHierarchy):
    """Compute compression metrics"""
    finest = hierarchy.get_level(hierarchy.finest_level)
    coarsest = hierarchy.get_level(hierarchy.coarsest_level)
    
    compression_ratio = finest.num_vertices() / coarsest.num_vertices()
    num_levels = hierarchy.num_levels()
    
    print(f"Compression Metrics:")
    print(f"  Original vertices: {finest.num_vertices()}")
    print(f"  Base domain vertices: {coarsest.num_vertices()}")
    print(f"  Compression ratio: {compression_ratio:.2f}x")
    print(f"  Hierarchy levels: {num_levels}")
    print(f"  Expected levels (log N): {math.log2(finest.num_vertices()):.1f}")

def plot_error_vs_level(hierarchy: MeshHierarchy):
    """Plot approximation error at each level"""
    # TODO: Implement error tracking and plotting
    pass

## 12. TODO List for Implementation

### Member 1: Geometry and Simplification
- [ ] Implement `compute_area_1ring()` - compute 1-ring area
- [ ] Implement `estimate_curvature()` - principal curvature estimation
- [ ] Complete `conformal_flatten_boundary()` - boundary case
- [ ] Implement `simplify_one_level()` - complete vertex removal logic
- [ ] Add proper 1-ring ordering in `get_1ring_ordered()`
- [ ] Implement boundary detection in `is_boundary_vertex()`

### Member 2: Parameterization
- [ ] Implement `_update_parameterization()` - track vertex mappings
- [ ] Handle three cases in parameterization update
- [ ] Implement triangle flipping detection and correction
- [ ] Store barycentric coordinates for all vertices
- [ ] Test parameterization quality

### Member 3: Progressive Encoding
- [ ] Implement `_encode_level()` - generate OBJA packets
- [ ] Track face modifications during simplification
- [ ] Implement vertex prioritization for encoding
- [ ] Add error bounds computation
- [ ] Test OBJA format validity

### Member 4: Integration and Testing
- [ ] Set up testing framework
- [ ] Implement all unit tests
- [ ] Create integration tests
- [ ] Test with provided datasets
- [ ] Submit to online evaluation platform
- [ ] Prepare presentation materials
- [ ] Document code

### All Members
- [ ] Weekly progress meetings
- [ ] Code reviews
- [ ] Debugging sessions
- [ ] Performance optimization
- [ ] Final presentation preparation